# MODELO 2 - SABENDO GENERO DO FILME

# POR ACABAR !!!!!!!!!!

In [38]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import warnings

#warnings.filterwarnings("ignore")

# Load the data

In [39]:
df_train = pd.read_csv("data/data_train.csv")
df_test = pd.read_csv("data/data_test.csv")

In [40]:
with open("data/data_ids.json") as f:
    ids = json.load(f)

max_movieId = max(ids["moviesIDs"].values())
max_userId = max(ids["userIDs"].values())

# Create a matrix of users and movies
base_matrix = np.zeros((max_movieId + 1, max_userId + 1))
base_matrix.shape

(9724, 610)

In [41]:
df_genre = pd.read_csv("data/movies.csv")

# Define functions

In [42]:
# Utility matrix and Binary matrix
from scipy.sparse import lil_matrix

def utility_matrix_AND_binary_matrix(dfTrain, zerosMatrix = base_matrix, jsonIds = ids):
    Y = lil_matrix(zerosMatrix.shape)  # Use sparse matrix format
    R = lil_matrix(zerosMatrix.shape)
    for i, row in dfTrain.iterrows():
        original_movieId, original_userId = int(row["movieId"]), int(row["userId"])
        movieId = jsonIds["moviesIDs"][str(original_movieId)]
        userId = jsonIds["userIDs"][str(original_userId)]
        Y[movieId, userId] = row["rating"]
        R[movieId, userId] = 1
    return Y, R

In [43]:
def cofiCostFunc_Xknown(params, X, Y, R, num_users, num_movies, num_features, Lambda):
    """
    Returns the cost and gradient for the collaborative filtering
    Lambda - regularization parameter
    """
        
    # Unfold the params
    Theta = params
    
    predictions =  X @ Theta.T
    err = (predictions - Y)
    J = 1/2 * np.sum((err**2) * R)
    
    #compute regularized cost function
    reg_Theta = Lambda/2 *np.sum(Theta**2)
    reg_J = J + reg_Theta
    
    # Compute gradient
    Theta_grad = (err*R).T @ X
    grad = Theta_grad
    
    # Compute regularized gradient
    reg_Theta_grad = Theta_grad + Lambda*Theta
    reg_grad = reg_Theta_grad
    
    return J, grad, reg_J, reg_grad

In [44]:
def gradientDescent_Xknown(initial_parameters,X,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda):
    """
    alpha - learning rate
    Optimize X and Theta
    """
    # unfold the initial parameters (consult function cofiCostFunc)
    Theta = initial_parameters
    
    J_history =[]
    
    for i in range(num_iters):
        #Append into a single vector params X and Theta (see above)
        params = Theta
        cost, Theta_grad = cofiCostFunc_Xknown(params, X, Y, R, num_users, num_movies, num_features, Lambda)[2:]
        
        #Update trainable parameters X & Theta applying gradient descent method        
        Theta = Theta - alpha*Theta_grad
        
        J_history.append(cost)
    
    #Append into a single vector paramsFinal the updated X and Theta
    paramsFinal = Theta
    return paramsFinal , J_history

# Set model features: X (movies genres as features)

In [45]:
num_movies, num_users = base_matrix.shape

genres = df_genre["genres"].str.split("|").tolist()
genres = {genre: 0 for sublist in genres for genre in sublist}
# create X: movies features
X = np.zeros((num_movies, len(genres))) # 20 genres
for movie in ids["moviesIDs"]:
    movieId_matrix = ids["moviesIDs"][movie]
    genre_dict = genres.copy()
    genres_list = df_genre[df_genre["movieId"] == int(movie)]["genres"].str.split("|").tolist()[0]
    for genre in genres_list:
        genre_dict[genre] = 1 / len(genres_list)
    X[movieId_matrix] = list(genre_dict.values())

X.shape

(9724, 20)

# Start modelling

In [46]:
# Shuffle the DataFrame randomly and split it into 8 parts
df_shuffled = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
folds = np.array_split(df_shuffled, 8)

# define the number of movies, users and features
num_movies, num_users = base_matrix.shape
num_features = 20

/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [47]:
Lambdas = [0, 5, 6, 7, 8, 10, 13, 15, 20, 25]
alphas = [0.0001, 0.0004, 0.0005, 0.00065, 0.00075, 0.001, 0.0015]
num_iters = 500

combo = 1
results = {}
for Lambda in Lambdas:
    for alpha in alphas:
        print(f"Combination {combo} of {len(Lambdas)*len(alphas)} with ({Lambda}, {alpha}):", end=" ") 
        combo += 1

        avg_RMSE = 0
        for exclude_index in range(8):
            merged_df = pd.concat([part for i, part in enumerate(folds) if i != exclude_index])

            # Create the utility matrix and the binary matrix
            Y, R = utility_matrix_AND_binary_matrix(merged_df)
            Y, R = Y.toarray(), R.toarray()

            # Define the initial parameters
            X = X
            Theta = np.random.normal(loc=0, scale=0.3, size=(num_users, num_features))
            initial_parameters = Theta

            # Apply gradient descent
            param, jhist = gradientDescent_Xknown(initial_parameters,X,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda)

            # Unfold the final parameters
            X = X
            Theta = param

            # Predict all ratings of num_users for num_movies
            p = X @ Theta.T

            # Compute the RMSE
            RMSE = 0
            for i, row in folds[exclude_index].iterrows():
                movieId_matrix = ids["moviesIDs"][str(int(row["movieId"]))]
                userId_matrix = ids["userIDs"][str(int(row["userId"]))]
                RMSE += abs(row["rating"] - p[int(movieId_matrix), int(userId_matrix)])**2
            RMSE = np.sqrt(RMSE/len(folds[exclude_index]))
            print(f"CV{exclude_index+1}: {RMSE:.1f}", end=", ")
            avg_RMSE += RMSE/8
        
        results[(Lambda, alpha)] = avg_RMSE
        print("")

Combination 1 of 70 with (0, 0.0001): CV1: 2.0, CV2: 2.0, CV3: 2.0, CV4: 2.0, CV5: 2.0, 

KeyboardInterrupt: 

In [36]:
results_stored = {(69,420): 0}
results

{(0, 0.0001): 3.5617476155629912,
 (0, 0.0004): 3.3647627713682944,
 (0, 0.0005): 3.308638072495282,
 (0, 0.00065): 3.2251508311610575,
 (0, 0.00075): 3.18005954686628,
 (0, 0.001): 3.0735950339766234,
 (0, 0.0015): 2.901344224418132,
 (5, 0.0001): 3.5704311530061683,
 (5, 0.0004): 3.366494569709523,
 (5, 0.0005): 3.30610964103165,
 (5, 0.00065): 3.2325470891541594}

In [ ]:
from scipy.interpolate import griddata

# Extract x, y, and z values
points = np.array(list(results.keys()))
x, y = points[:, 0], points[:, 1]
z = np.array(list(results.values()))

# Create a grid for interpolation
xi = np.linspace(min(x), max(x), 100)
yi = np.linspace(min(y), max(y), 100)
xi, yi = np.meshgrid(xi, yi)

# Interpolate z values onto the grid
zi = griddata(points, z, (xi, yi), method='cubic') #nearest, cubic, linear

# Plot the topographic map
plt.figure(figsize=(8, 6))
contour = plt.contourf(xi, yi, zi, levels=50, cmap='viridis')
plt.colorbar(contour, label='8-Fold Cross-Validation RMSE (500 iterations)')
best_lambda, best_alpha = 6, 0.0005
plt.scatter(best_lambda, best_alpha, color='red', s=20)
plt.text(
    best_lambda * 1.15, 
    best_alpha, 
    r"$\lambda$ = " + f"{best_lambda}\n" + r"$\alpha$ = " + str(best_alpha), 
    fontsize=13.5, 
    color='black',
    va = 'center',
    bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.2', alpha=0.6)
)
plt.title('Fine-Tuning HSDUIAHSDIUHASDIUHASDI Hyperparameters')
plt.xlabel(r'Regularization Parameter ($\lambda$)')
plt.ylabel(r'Learning Rate ($\alpha$)')
plt.show()

In [37]:
# choose the best hyperparameters
best_lambda, best_alpha = 69, 420
print(f"The best hyperparameters are: ({best_lambda}, {best_alpha}) with RMSE: {results_stored[(best_lambda, best_alpha)]:.5f}")

The best hyperparameters are: (69, 420) with RMSE: 0.00000


## ASDASD



In [2]:
"""num_features = len(genres)
Theta = np.random.normal(loc=0, scale=0.3, size=(num_users, num_features))
Y, R = utility_matrix_AND_binary_matrix(df_train)
Y, R = Y.toarray(), R.toarray()
Lambda = 1

params = Theta"""

'num_features = len(genres)\nTheta = np.random.normal(loc=0, scale=0.3, size=(num_users, num_features))\nY, R = utility_matrix_AND_binary_matrix(df_train)\nY, R = Y.toarray(), R.toarray()\nLambda = 1\n\nparams = Theta'

In [1]:
"""initial_parameters = Theta
alpha = 0.001
num_iters = 500
a,b = gradientDescent_Xknown(initial_parameters,X,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda)
plt.plot(b)"""

'initial_parameters = Theta\nalpha = 0.001\nnum_iters = 500\na,b = gradientDescent_Xknown(initial_parameters,X,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda)\nplt.plot(b)'